## Keras and TF to build an LSTM

### Text generation from Macbeth.txt file. Example from https://www.analyticsvidhya.com/blog/2017/12/fundamentals-of-deep-learning-introduction-to-lstm/

In [24]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

In [25]:
filename = "macbeth.txt"

text = (open(filename).read()).lower()

# mapping characters with integers
unique_chars = sorted(list(set(text)))

#create empty dictionaries
char_to_int = {}
int_to_char = {}

#creating character to integer mappings
for i, c in enumerate (unique_chars):
    char_to_int.update({c: i})
    int_to_char.update({i: c})
    

In [26]:
# preparing input and output dataset
X = []
Y = []

# working with chunks of 50 characters. Use the first 49 characters to train on, and the 50th as the labelA
for i in range(0, len(text) - 50, 1):
    sequence = text[i:i + 50]
    label =text[i + 50]
    X.append([char_to_int[char] for char in sequence])
    Y.append(char_to_int[label])


#### A LSTM network expects the input to be in the form [samples, time steps, features] where samples is the number of data points we have, time steps is the number of time-dependent steps that are there in a single data point, features refers to the number of variables we have for the corresponding true value in Y. We then scale the values in X_modified between 0 to 1 and one hot encode our true values in Y_modified.



In [27]:
# reshaping, normalizing and one hot encoding
X_modified = np.reshape(X, (len(X), 50, 1))
X_modified = X_modified / float(len(unique_chars))
Y_modified = np_utils.to_categorical(Y)

In [28]:
#defining the LSTM model
model = Sequential()
model.add(LSTM(units = 300, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(rate = 0.2))
model.add(LSTM(units = 300))
model.add(Dropout(rate = 0.2))
model.add(Dense(units = Y_modified.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')


In [31]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 50, 300)           362400    
_________________________________________________________________
dropout_7 (Dropout)          (None, 50, 300)           0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 300)               721200    
_________________________________________________________________
dropout_8 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 43)                12943     
Total params: 1,096,543
Trainable params: 1,096,543
Non-trainable params: 0
_________________________________________________________________


In [32]:
# fitting the model
model.fit(X_modified, Y_modified, epochs=2, batch_size=30)

Epoch 1/2
100271/100271 [==============================] - 697s 7ms/step - loss: 2.7484
Epoch 2/2
100271/100271 [==============================] - 740s 7ms/step - loss: 2.4903


In [33]:
# picking a random seed
start_index = np.random.randint(0, len(X)-1)
new_string = X[start_index]

# generating characters
for i in range(50):
    x = np.reshape(new_string, (1, len(new_string), 1))
    x = x / float(len(unique_chars))

    #predicting
    pred_index = np.argmax(model.predict(x, verbose=0))
    char_out = int_to_char[pred_index]
    seq_in = [int_to_char[value] for value in new_string]
    print(char_out)

    new_string.append(pred_index)
    new_string = new_string[1:len(new_string)]


m
a
c
d
.
 
w
h
e
 
s
h
e
 
t
h
e
 
t
h
e
e
e
 
t
h
e
 
s
h
e
e
e
 
t
h
e
 
s
h
e
e
e
 
t
h
e
 
s
h
